In [57]:
%run import.ipynb

(14800, 41)


In [58]:
import xgboost as xgb
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

----

In [59]:
df.columns

Index(['AAGE', 'ACLSWKR', 'ADTIND', 'ADTOCC', 'AHGA', 'AHRSPAY', 'AHSCOL',
       'AMARITL', 'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM',
       'AUNTYPE', 'AWKSTAT', 'CAPGAIN', 'CAPLOSS', 'DIVVAL', 'FILESTAT',
       'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MARSUPWT', 'MIGMTR1',
       'MIGMTR3', 'MIGMTR4', 'MIGSAME', 'MIGSUN', 'NOEMP', 'PARENT',
       'PEFNTVTY', 'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'SEOTR', 'TAXINC',
       'VETQVA', 'VETYN', 'WKSWORK'],
      dtype='object')

In [60]:
# vars_to_drop = ['AWKSTAT', 'WKSWORK']
# df.drop(vars_to_drop, axis=1, inplace=True)

----

In [61]:
cat_variables = [i for i in df.columns if df[i].dtype == 'O']
print(cat_variables)

['ACLSWKR', 'AHGA', 'AHSCOL', 'AMARITL', 'AMJIND', 'AMJOCC', 'ARACE', 'AREORGN', 'ASEX', 'AUNMEM', 'AUNTYPE', 'AWKSTAT', 'FILESTAT', 'GRINREG', 'GRINST', 'HHDFMX', 'HHDREL', 'MIGMTR1', 'MIGMTR3', 'MIGMTR4', 'MIGSAME', 'MIGSUN', 'PARENT', 'PEFNTVTY', 'PEMNTVTY', 'PENATVTY', 'PRCITSHP', 'TAXINC']


In [42]:
# encods = [LabelEncoder() for col in cat_variables]
# for i, col in enumerate(cat_variables):
#     df[col] = encods[i].fit_transform(df[col].astype(str))

In [62]:
dummies = cat_variables
# df_cat = df.loc[:, dummies]
for c in dummies:
    temp = pd.get_dummies(df[c], dummy_na=True, prefix=c)
    df = pd.merge(df_cat, temp, on=df.index)
    df.drop(['key_0'], axis=1, inplace=True)

IndexError: indices are out-of-bounds

----

In [43]:
df.fillna(0, inplace=True)

------

In [44]:
X = df[:10000]
X_test = df[10000:]
print(X.shape)
print(X_test.shape)

(10000, 41)
(4800, 41)


----

In [45]:
y.value_counts()

-50000    5000
50000+    5000
Name: CLASS, dtype: int64

In [46]:
y.replace({'-50000' : 0, '50000+' : 1}, inplace=True)

----

In [47]:
dtrain = xgb.DMatrix(X, y, missing=0.0)
dtest = xgb.DMatrix(X_test)

ValueError: DataFrame.dtypes for data must be int, float or bool.
Did not expect the data types in fields ACLSWKR, AHGA, AHSCOL, AMARITL, AMJIND, AMJOCC, ARACE, AREORGN, ASEX, AUNMEM, AUNTYPE, AWKSTAT, FILESTAT, GRINREG, GRINST, HHDFMX, HHDREL, MIGMTR1, MIGMTR3, MIGMTR4, MIGSAME, MIGSUN, PARENT, PEFNTVTY, PEMNTVTY, PENATVTY, PRCITSHP, TAXINC

In [ ]:
param = { 'objective' : "binary:logistic" }
num_round = 100
bst = xgb.train(param, dtrain, num_round)

In [ ]:
pred = bst.predict(dtest)

In [ ]:
pred = pred.round()

In [ ]:
filename = 'submissions/xgb_' + datetime.now().strftime(format = '%d%m_%H%M') + '.csv'
pd.DataFrame({'ID':ids, 'Prediction': pred}).to_csv(filename, index=False, sep=',')